# KHARMA Tutorial 
# CCA Numerical Series on Fluids and Plasmas (10/24/2024)
---
---

## Getting KHARMA

KHARMA can be found on Github [here](https://github.com/AFD-Illinois/kharma).
For this tutorial we'll be working with the `tutorial` branch which can be cloned like,

`git clone -b tutorial https://github.com/AFD-Illinois/kharma.git`

---

## Building pyharm

Additionally, we'll need `pyharm`, a Python package for analyzing GRMHD data products from HARM-based codes. The code is hosted [here](https://github.com/AFD-Illinois/pyharm), and can be obtained using git,

`git clone https://github.com/AFD-Illinois/pyharm.git`

It's generally good practice to isolate Python environments when working on different projects to avoid dependency hell. Also, the SCC at FI won't let us install python packages on the Python moduel provided by them. Let's download the python module (if you don't have it by default in your module stack), set up a virtual environment, and install `pyharm`,

```
module load python3
VENVDIR=PATH_WHERE_PYTHON_VENVS_ARE_STORED
python3 -m venv --system-site-packages $VENVDIR/pyharm
source $VENVDIR/pyharm/bin/activate
cd pyharm
pip3 install -e .
```
where, `VENVDIR` is the location where you would want to save your Python environments; for this tutorial we've created a virtual enviroment `pyharm`. More about Python virtual environments on Rusty [here](https://wiki.flatironinstitute.org/SCC/Software/PythonVirtualEnvironments). 

If you instead prefer working with `conda`, `pyharm` has a `environment.yml` file that can be used to set up an environment with `conda`. More about that [here](https://pyharm.readthedocs.io/en/latest/installing.html).

---

## Building KHARMA

KHARMA has two main dependencies which must be checked out before compiling it,
```
cd kharma
git submodule update --init --recursive
```

KHARMA has an extensive [wiki](https://github.com/AFD-Illinois/kharma/wiki) and lists the minimum requirements needed. Let us set up the module environment needed to compile KHARMA for GPUs on Rusty,

```
module load modules/2.3-20240529 slurm cmake gcc/11.4.0 cuda gsl openmpi hdf5 openblas
```
For CPUs, it's nearly the same, except we don't need the `cuda` module. 

KHARMA uses `cmake` to build the code called from the `make` shell script. This script in conjuction with machine-specific build files are help set up the module environment and pass necessary arguments to `cmake`. By default KHARMA parallelizes the build process, and therefore it is good practice to compile KHARMA on a compute node. To get an interactive session on a compute node on Rusty try,

`srun -N 1 --constraint=ib-icelake --pty bash -i`

To run KHARMA on NVIDIA A100s use the following command,

`./make.sh clean cuda a100`

The `cuda` argument tells Kokkos to build with CUDA, while the `a100` argument specifies the device architecture. KHARMA's build philosophy is to have separate machine-specific files that setup the relevant module environment and environment variables, e.g., there is a machine config file for Rusty (`machines/rusty.sh`) that builds for different architectures available on Rusty.

If you'd like to build for KHARMA to be run on CPU, say the icelake nodes, the build command would be,

`./make.sh clean skx`.

At the end, you should see the following output on the screen that comfirms KHARMA was built successfully,

![gpu build](images/screenshot_gpu_build.png)

OR

![cpu build](images/screenshot_cpu_build.png)

depending on the type of build. The postfix (`.cuda`, `.host`) in the executable name specifies the architecture KHARMA was built for. More about compiling [here](https://github.com/AFD-Illinois/kharma/wiki/Building-KHARMA).

---

## Running KHARMA

KHARMA can either be run in broadly two ways,
- By directly calliing the executable: `./kharma.host -i PROBLEM_PARAMETER_FILE`. This works best when you are running on a personal computer or are on a compute node via an interactive job, and have manually set the module stack and required environment variables correctly.

- Invoking `run.sh` or a batch script: Often, we don't want to manually set the environment and want the runtime environment to be set automatically based on the compile-time options. This is where `run.sh` or a machine-specific batch script comes in.

For this tutorial we'll use the slurm batch scripts located in the `scripts/tutorial/batch` directory. This script sets the SLURM directives, sources the machine file to set the module environment, and submits the job.

### Problem 1: Pressure equilibrium in ideal MHD

It is good practice to create a separate directory for each problem run; say `SIMULATION_DIR` is where we'll run the first problem. KHARMA will output data to the directory where the executable is called from. Let's run the following set of commands,

```
cd SIMULATION_DIR
cp PATH_TO_KHARMA_SOURCE/scripts/tutorial/batch/rusty_gpu.sb ./
cp PATH_TO_KHARMA_SOURCE/scripts/tutorial/parfiles/anisotropic_conduction_ideal.par ./
```
This copies over the job submit script and the parameter file for the problem to `SIMULAITON_DIR`. Before we move on to running the problem, let's take a look at the parameter file.

KHARMA was designed from ground up with flexibility and extensibility in mind; most parameters are therefore runtime. This means the number of parameters is [long and growing](https://github.com/AFD-Illinois/kharma/wiki/Parameters). However, on a day-to-day basis there are only a limited number of parameters that you'd need to modify depending on the problem and KHARMA sets sensible default values for the rest.
Parameters are divided into blocks and are specified like `block_name/parameter_name=value`. Let's take a look at the parameter file for this problem to see this in practice.

Now, let's submit the job

`sbatch rusty_gpu.sb -i anisotropic_conduction_ideal.par`

In this case we explicitly ask Parthenon (via the `-d` flag in the job submit script) to create the save output (dump files, restart files, etc..) in a separate directory `dumps_kharma`. We could have pointed to the parameter file at `PATH_TO_KHARMA_SOURCE/scripts/tutorial/parfiles/` but it is cleaner this way---each simulation has its own copy of the parameter file and we avoid changing the defaults in the git repository.

**Congratulations on running your first KHARMA simulation!**

Let's take a look at the output. The standard output is saved to the `.out` file. When the problem is executing, you'll see a line like this (for explicitly-evolved problems at least),

```
cycle=50 time=6.0457029249727713e-02 dt=1.4976930529853696e-03 zone-cycles/wsec_step=4.24e+07 wsec_total=4.25e-01 wsec_step=6.18e-03
Max DivB: 1.52656e-16
```
`cycle` denotes the timestep number, `time` is the simulation time in code units (e.g., for a black hole accretion problem it'd be in units of the light-crossing time $GM/c^3$), `dt` is the size of the timestep, and `zone-cycles/wsec_step` is the number of zone cycle per second.

The simulation output is saved to `dump_kharma`. Files ending with `phdf` are the fluid "dump" files and stores the state of the fluid at a given time. Restart files have the `rhdf` extensions.

It is most convenient to use `pyharm` to work with KHARMA output. At the heart of `pyharm` is the FluidDump object that caches the data in the dump file, and computes derived quantities from the cached data on-the-fly. It acts like a Python dictionary from which various quantities (related to the fluid state, grid, input parameters, etc..) can be accessed by passing the right key. The `load_dump` method is used to obtain the fluid state object. For example,

In [ ]:
import pyharm
import matplotib.pyplot as plt

dump = pyharm.load_dump(os.path.join(dumpsdir, 'anisotropic_conduction.out0.00000.phdf'))

fig = plt.figure()
ax  = fig.gca()
ax.pcolormesh(np.squeeze(dump['X1']), np.squeeze(dump['X2']), np.squeeze(dump['Theta']), cmap = 'viridis', shading='gouraud')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_xticks([0,0.25,0.5,0.75,1])
ax.set_xticklabels([0,0.25,0.5,0.75,1])
ax.set_yticks([0,0.25,0.5,0.75,1])
ax.set_yticklabels([0,0.25,0.5,0.75,1])
ax.set_xlabel('$x (GM/c^2)$')
ax.set_ylabel('$y (GM/c^2)$')
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(temp_plot, cax=cax)
cbar.set_label('P$_{g}/\\rho$', rotation=90)

plt.show()